In [2]:
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *

In [3]:
# 辅助函数
# 栅格转面
def raster_to_polygon(in_raster, out_polygon, field_name):
    """栅格转面"""
    in_raster = arcpy.sa.Int(in_raster)
    output_coordinate_system = arcpy.Describe(in_raster).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system):
        arcpy.conversion.RasterToPolygon(in_raster, out_polygon, "SIMPLIFY", f"{field_name}", "SINGLE_OUTER_PART", None)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

In [4]:
# 构建转换字典
table_path = r"C:\Users\Runker\Desktop\dy_0417.csv"

In [5]:
df = pd.read_csv(table_path)

In [6]:
# 对每个唯一的土种编号生成一个字典
def aggregate_rows(group):
    records = group[['TL', 'YL', 'TS']].to_dict('records')
    # 假设每个土种编号下的记录是相同的，这里我们取第一个记录
    return records[0] if records else {}

# 使用groupby按照`土种编号`聚合，并应用自定义函数
soil_dict = df.groupby('TZ').apply(aggregate_rows).to_dict()

In [7]:
tz_dict = {0: ['中白鳝泥田'], 1: ['中腐殖层中土层硅质黄壤'], 2: ['中腐殖层中土层黄色石灰土'], 3: ['中腐殖层厚土层硅质黄壤'], 4: ['中腐殖层薄土层硅质黄壤'], 5: ['中腐殖层薄土层黄色石灰土'], 6: ['偏沙土'], 7: ['冷水田'], 8: ['冷沙土'], 9: ['冷沙田'], 10: ['冷浸田'], 11: ['大泥土'], 12: ['大泥田'], 13: ['大眼泥田'], 14: ['大眼黄泥田'], 15: ['小泥土'], 16: ['小黄泥土'], 17: ['小黄泥田'], 18: ['岩泥'], 19: ['干鸭屎泥田'], 20: ['扁沙泥田'], 21: ['死黄泥土'], 22: ['死黄泥田'], 23: ['浅足烂泥田'], 24: ['深足烂泥田'], 25: ['湿鸭屎泥田'], 26: ['潮板沙田'], 27: ['潮沙泥田'], 28: ['潮沙田'], 29: ['潮油沙泥田'], 30: ['潮泥田'], 31: ['火石沙土'], 32: ['烂锈田'], 33: ['熟鸭屎泥田'], 34: ['白云沙土'], 35: ['白沙土'], 36: ['白沙泥田'], 37: ['石碴子土'], 38: ['砾质紫沙土'], 39: ['紫沙土'], 40: ['紫色土'], 41: ['苦大泥田'], 42: ['薄腐殖中土层山地灌丛草甸土'], 43: ['薄腐殖层中土层山地灌丛草甸土'], 44: ['薄腐殖层中土层次生黄色石灰土'], 45: ['薄腐殖层中土层硅质黄壤'], 46: ['薄腐殖层中土层硅铁质黄壤'], 47: ['薄腐殖层中土层硅铝质黄壤'], 48: ['薄腐殖层中土层紫色土'], 49: ['薄腐殖层中土层铁铝质黄壤'], 50: ['薄腐殖层中土层黄色石灰土'], 51: ['薄腐殖层中土层黑色石灰土'], 52: ['薄腐殖层厚土层山地灌丛草甸土'], 53: ['薄腐殖层厚土层次生黄色石灰土'], 54: ['薄腐殖层厚土层硅质黄壤'], 55: ['薄腐殖层厚土层硅铁质黄壤'], 56: ['薄腐殖层厚土层硅铝质黄壤'], 57: ['薄腐殖层厚土层铁铝质黄壤'], 58: ['薄腐殖层厚土层黄色石灰土'], 59: ['薄腐殖层厚土层黑色石灰土'], 60: ['薄腐殖层薄土层山地灌丛草甸土'], 61: ['薄腐殖层薄土层次生黄色石灰土'], 62: ['薄腐殖层薄土层淋溶黄色石灰土'], 63: ['薄腐殖层薄土层硅质黄壤'], 64: ['薄腐殖层薄土层硅铁质黄壤'], 65: ['薄腐殖层薄土层硅铝质黄壤'], 66: ['薄腐殖层薄土层紫色土'], 67: ['薄腐殖层薄土层酸性紫色土'], 68: ['薄腐殖层薄土层铁质黄壤'], 69: ['薄腐殖层薄土层铁铝质黄壤'], 70: ['薄腐殖层薄土层黄壤性土'], 71: ['薄腐殖层薄土层黄色石灰土'], 72: ['薄腐殖层薄土层黑色石灰土'], 73: ['薄腐殖薄土层山地灌丛草甸土'], 74: ['重白胶泥田'], 75: ['重白鳝泥田'], 76: ['钙石子土'], 77: ['铅毒田'], 78: ['锈水田'], 79: ['雾黄泥田'], 80: ['马血泥田'], 81: ['黄大泥田'], 82: ['黄沙泥土'], 83: ['黄沙泥田'], 84: ['黄油沙泥田'], 85: ['黄泥土'], 86: ['黄泥田'], 87: ['黄石砂田'], 88: ['黄胶泥土'], 89: ['黄胶泥田'], 90: ['黑潮泥田'], 91: ['龙凤大眼泥田']}

In [8]:
result_dict = {}
for one_key in tz_dict:
    result_dict[one_key] = {'TZ':tz_dict[one_key][0],'TL':soil_dict[tz_dict[one_key][0]]['TL'],'TS':soil_dict[tz_dict[one_key][0]]['TS'],'YL':soil_dict[tz_dict[one_key][0]]['YL']}

In [9]:
result_dict

{0: {'TZ': '中白鳝泥田',
  'TL': '水稻土',
  'TS': '白泥田A—P E—E(C)或A—P—E(C)',
  'YL': '侧渗型水稻土'},
 1: {'TZ': '中腐殖层中土层硅质黄壤', 'TL': '黄壤', 'TS': '硅质黄壤', 'YL': '黄壤'},
 2: {'TZ': '中腐殖层中土层黄色石灰土', 'TL': '石灰土', 'TS': '黄色石灰土', 'YL': '黄色石灰土'},
 3: {'TZ': '中腐殖层厚土层硅质黄壤', 'TL': '黄壤', 'TS': '硅质黄壤', 'YL': '黄壤'},
 4: {'TZ': '中腐殖层薄土层硅质黄壤', 'TL': '黄壤', 'TS': '硅质黄壤', 'YL': '黄壤'},
 5: {'TZ': '中腐殖层薄土层黄色石灰土', 'TL': '石灰土', 'TS': '黄色石灰土', 'YL': '黄色石灰土'},
 6: {'TZ': '偏沙土', 'TL': '黄壤', 'TS': '石砂土', 'YL': '黄泥土'},
 7: {'TZ': '冷水田', 'TL': '水稻土', 'TS': '冷水田', 'YL': '潴育型水稻土'},
 8: {'TZ': '冷沙土', 'TL': '黄壤', 'TS': '石砂土', 'YL': '黄泥土'},
 9: {'TZ': '冷沙田', 'TL': '水稻土', 'TS': '冷浸田APG或APWC', 'YL': '潜育型水稻土'},
 10: {'TZ': '冷浸田', 'TL': '水稻土', 'TS': '冷浸田APG或APWC', 'YL': '潜育型水稻土'},
 11: {'TZ': '大泥土', 'TL': '石灰土', 'TS': '大土泥', 'YL': '大土泥'},
 12: {'TZ': '大泥田', 'TL': '水稻土', 'TS': '淹育型石灰性大泥田APC', 'YL': '淹育型水稻土'},
 13: {'TZ': '大眼泥田', 'TL': '水稻土', 'TS': '潴育型石灰性大眼泥田ARWC或APBC', 'YL': '潴育型水稻土'},
 14: {'TZ': '大眼黄泥田', 'TL': '水稻土', 'TS': '潴育型石灰性大眼泥田A

In [10]:
raster_path = r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_dy.gdb\prediction_soiltype_RFE_20240418_reclass_smooth'
out_polygon_path = r'D:\ArcGISProjects\workspace\shbyq\soil_type_result.gdb\soiltype_dy\prediction_soiltype_RFE_20240418_reclass_smooth'
field_name = 'Value'

In [11]:
# 栅格转面
raster_to_polygon(raster_path, out_polygon_path, field_name)

In [20]:
code_block =f"""
def get_soil_class(soil_code):
    soil_dict = {result_dict}
    TZ = soil_dict[soil_code]['TZ']
    TL = soil_dict[soil_code]['TL']
    YL = soil_dict[soil_code]['YL']
    TS = soil_dict[soil_code]['TS']
    result_list = [TZ,TS,YL,TL]
    return result_list[3]
"""
field_name = 'TL'

In [13]:
expression = "get_soil_class(!gridcode!)"

In [14]:
# 增加字段
for one_field in ['TL','YL','TS','TZ']:
    add_field(out_polygon_path, one_field, 'TEXT')

In [21]:
# 计算字段
arcpy.CalculateField_management(in_table=out_polygon_path, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)
    

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\soil_type_result.gdb\\soiltype_dy\\prediction_soiltype_RFE_20240418_reclass_smooth'>